In [1]:
# import libraries
import os
from typing import TypedDict, List, Dict
from langchain_groq import ChatGroq
from langchain.chains import LLMChain
from langchain.prompts import ChatPromptTemplate
from dotenv import load_dotenv

In [2]:
# defining categories
class Category(TypedDict):
    name: str
    keywords: List[str]
    response: str

categories: List[Category] = [
    {
        "name": "Billing",
        "keywords": ["payment", "invoice", "bill", "charge"],
        "response": "For billing questions, please check your account settings or contact billing support."
    },
    {
        "name": "Technical Support",
        "keywords": ["error", "problem", "not working", "issue"],
        "response": "Please try restarting your device. If the problem persists, contact technical support."
    },
    {
        "name": "General Inquiry",
        "keywords": ["hours", "location", "contact"],
        "response": "Our business hours are 9 AM to 5 PM, Monday to Friday."
    }
]

In [3]:
# classification
def classify_question(question: str, categories: List[Category]) -> str:
    question_lower = question.lower()
    category_scores: Dict[str, int] = {}

    for category in categories:
        score = sum(keyword in question_lower for keyword in category["keywords"])
        category_scores[category["name"]] = score

    best_match = max(category_scores, key=category_scores.get)

    if category_scores[best_match] == 0:
        return "Unknown"

    return best_match

In [4]:
# connect to groq
load_dotenv()
API_KEY = os.getenv("OPENAI_API_KEY")

llm = ChatGroq(model="llama-3.1-8b-instant")

# Quick test
test_response = llm.invoke("Hello! Can you confirm you are working?")
print(test_response.content)

Hello! Yes, I am working and ready to assist you with any questions or topics you'd like to discuss. How can I help you today?


In [9]:
# prompt template & chain
prompt = ChatPromptTemplate.from_template("""
You are a helpful support assistant.
Classify the question into one of these categories:
- Billing
- Technical Support
- General Inquiry

Then respond with a short, polite message.
Question: {question}
""")

chain = prompt | llm

test_output = chain.invoke({"question": "My payment is not going through, what should I do?"})
print(test_output.content)

Category: Billing

I'd be happy to help you with your payment issue. Can you please try processing the payment again and check if there are any error messages? If the issue persists, please provide me with your order number or account information so I can assist you further.


In [6]:
# frustration detection
frustration_keywords = ["angry", "frustrated", "not happy"]

def check_frustration(text: str) -> bool:
    return any(word in text.lower() for word in frustration_keywords)

In [7]:
def format_history(chat_history):
    """
    Turns chat_history list into a single formatted string.
    Example:
    User: Hello
    Bot: Hi there!
    """
    return "\n".join(f"{speaker.capitalize()}: {text}" for speaker, text in chat_history)

def run_chatbot():
    chat_history = []

    while True:
        user_input = input("You: ")
        if user_input.lower() in ["exit", "quit"]:
            print("Bot: Goodbye!")
            chat_history.append(("bot", "Goodbye!"))
            break

        chat_history.append(("user", user_input))

        if check_frustration(user_input):
            category = classify_question(user_input, categories)
            response = f"I can see this is frustrating. I'll connect you to a human in {category} support."
            print(f"Bot: {response}")
            chat_history.append(("bot", response))
            continue

        category = classify_question(user_input, categories)

        if category == "Unknown":
            # Pass chat history to the LLM for better context
            full_context = format_history(chat_history)
            llm_prompt = (
                f"The following is a conversation between a user and a helpful chatbot.\n"
                f"{full_context}\n"
                f"Bot:"
            )
            llm_response = chain.invoke({"question": llm_prompt})
            response = llm_response.content
        else:
            response = next(c["response"] for c in categories if c["name"] == category)

        print(f"Bot: {response}")
        chat_history.append(("bot", response))

    print("\n--- Chat History ---")
    for speaker, text in chat_history:
        print(f"{speaker.capitalize()}: {text}")


# Start chatbot
run_chatbot()


Bot: Classification: Technical Support

Response:
"I'm so sorry to hear that our page is causing you difficulties. I'd be happy to help you troubleshoot the issue. Can you please tell me more about the problems you're experiencing, such as the speed of the page load or any error messages you've seen? This will help me provide a more accurate solution."
Bot: Classification: Technical Support

Response:
"I understand that the page load time is a major issue for you. Can you please tell me what type of device and browser you're using to access our page? Additionally, have you recently updated your browser or tried clearing your cache and cookies? This information will help me narrow down the possible causes of the slow page load."
Bot: Category: Technical Support

Response:
"I appreciate your patience and for providing more details. Since the issue is occurring on multiple devices and browsers, it might be related to our server or network. I'm going to escalate this to our technical team 